# CHECKLIST
* Fixa dataloaders
    * Vafan är datan egentligen?
    * 
* Fixa model
    * Jag antar vi kör CNN (CSN plz)
    * Prova andra arkitekturer?
    *
* Fixa hyperparametrar, kanske använda maskininlärning för det?
* Fixa CUDA support och träna hemma?
* Fixa det sista med matrisen


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import copy
import json
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import netCDF4
import netCDF4 as nc
import xarray as xr
import pandas as pd
from netCDF4 import Dataset

import os



ModuleNotFoundError: No module named 'seaborn'

In [123]:
DATA_PATH = r"/Users/ludvigflodin/Code/ml-cloud-opt-thick-master/Tjena/skogsstyrelsen-data"
BASE_PATH_LOG = '../log'
BASE_PATH_DATA = '../data/skogsstyrelsen'
SPLIT_TO_USE = 'trainval' 

# Read data + corresponding json info (incl ground truth)
img_paths_train = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_names_train.npy')))
img_paths_val = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_names_val.npy')))
img_paths_test = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_names_test.npy')))
json_content_train = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_json_train.npy'), allow_pickle=True))
json_content_val = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_json_val.npy'), allow_pickle=True))
json_content_test = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_json_test.npy'), allow_pickle=True))
# Run model on desired split

if SPLIT_TO_USE == 'train':
	img_paths = img_paths_train
	json_paths = json_content_train
elif SPLIT_TO_USE == 'val':
	img_paths = img_paths_val
	json_paths = json_content_val
elif SPLIT_TO_USE == 'trainval':
	img_paths = img_paths_train + img_paths_val
	json_paths = json_content_train + json_content_val
elif SPLIT_TO_USE == 'test':
	img_paths = img_paths_test
	json_paths = json_content_test
all_binary_preds = []
all_binary_gts = []
for img_idx, img_path in enumerate(img_paths):

	print(img_idx, len(img_paths))

	# Extract date to see if data is from before or after Jan 2022
	# (this affects the normalization used for the image)
	img = xr.open_dataset(img_path)
	yy_mm_dd = getattr(img, 'time').values[0]
	yy = yy_mm_dd.astype('datetime64[Y]').astype(int) + 1970
	mm = yy_mm_dd.astype('datetime64[M]').astype(int) % 12 + 1

FileNotFoundError: [Errno 2] No such file or directory: '../data/skogsstyrelsen/skogs_names_train.npy'

In [16]:
import os
import xarray as xr
from torch.utils.data import Dataset
import numpy as np

#Creates a dataset for our cloud images
class CloudClassificationDataset(Dataset):
    def init(self, imagedir, json, transform=None, channels:tuple=("b04","b03","b02")):
        self.image_dir = image_dir
        self.transform = transform
        self.channels = channels
        self.json = np.load(json,allow_pickle=True)

    def len(self):
        return len(self.json)

    def getitem(self, index):
        # Finds filename in json file and creates path to corresponding image
        filename = "skgs"+self.json[index]['ValideringsobjektBildId']+".nc"
        img_path = os.path.join(self.image_dir,filename)

        # Opens NetCDF4 image
        df = xr.open_dataset(img_path, engine='netcdf4')

        # Extracts labels from the json file
        label = self.json[index]["MolnDis"]

        #Creates a tuple of dataArray objects with given channels from the tuple of channels and the dataArray
        channel_lst=[]
        for c in self.channels:
            channel_lst.append(df[c][0])
        channel_tup = tuple(channel_lst)

        # Stacks, normalizes, and cut the image to a (20, 20, C) format between 0 and 1
        imagergb = np.dstack(channel_tup)
        cli=0
        image = np.clip((imagergb-np.percentile(imagergb,cli))/(np.percentile(imagergb,100-cli)-np.percentile(imagergb,cli)),0, 1)[:20,:20,:]

        return image,int(label)

In [17]:
#import CloudClassificationDataset
import torch
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from PIL import Image
batch_size = 3

In [18]:
##Get the dataset of the CloudDataset
dataset = CloudClassificationDataset.CloudClassificationDataset(r"C:\Users\affel\D7046E - Course\ml-cloud-opt-thick-master\Tjena\data\skogsstyrelsen\skogs_json_train.npy")

##Creates validation/training dataset with 20/80 split
validation_dataset, training_dataset = torch.utils.data.random_split(dataset,[0.2, 0.8])

##Create a DataLoaders from the datasets.
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True)

##Creates an iterator and plots the pictures
it = iter(training_loader)
images, labels = next(it)

plt.figure()
plt.imshow(images[0])
plt.show()

print(labels[0].item())

AttributeError: type object 'CloudClassificationDataset' has no attribute 'CloudClassificationDataset'

In [19]:
import os
import xarray as xr
from torch.utils.data import Dataset
import numpy as np

#Creates a dataset for our cloud images
class CloudClassificationDataset(Dataset):
    def __init__(self, imagedir, json, transform=None, channels=("b04", "b03", "b02")):
        self.image_dir = imagedir
        self.transform = transform
        self.channels = channels
        self.json = np.load(json, allow_pickle=True)

    def __len__(self):
        return len(self.json)

    def __getitem__(self, index):
        # Finds filename in json file and creates path to corresponding image
        filename = "skgs" + self.json[index]['ValideringsobjektBildId'] + ".nc"
        img_path = os.path.join(self.image_dir, filename)

        # Opens NetCDF4 image
        df = xr.open_dataset(img_path, engine='netcdf4')

        # Extracts labels from the json file
        label = self.json[index]["MolnDis"]

        # Creates a tuple of dataArray objects with given channels from the tuple of channels and the dataArray
        channel_lst = []
        for c in self.channels:
            channel_lst.append(df[c][0])
        channel_tup = tuple(channel_lst)

        # Stacks, normalizes, and cut the image to a (20, 20, C) format between 0 and 1
        imagergb = np.dstack(channel_tup)
        cli = 0
        image = np.clip((imagergb - np.percentile(imagergb, cli)) / (
                    np.percentile(imagergb, 100 - cli) - np.percentile(imagergb, cli)), 0, 1)[:20, :20, :]

        return image, int(label)

# Path to your dataset
image_dir = r"C:\Users\affel\D7046E - Course\ml-cloud-opt-thick-master\Tjena\data\skogsstyrelsen"
json_path = r"C:\Users\affel\D7046E - Course\ml-cloud-opt-thick-master\Tjena\data\skogsstyrelsen\skogs_json_train.npy"

# Create an instance of your dataset
dataset = CloudClassificationDataset(image_dir, json_path)

# Define batch size
batch_size = 3

# Split the dataset into validation and training sets
validation_size = int(0.2 * len(dataset))
training_size = len(dataset) - validation_size
validation_dataset, training_dataset = torch.utils.data.random_split(dataset, [validation_size, training_size])

# Create DataLoader instances for validation and training
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True)

# Creates an iterator and plots the pictures
it = iter(training_loader)
images, labels = next(it)

# Display the first image in the batch
plt.figure()
plt.imshow(images[0].numpy().transpose((1, 2, 0)))  # Transpose image to (H, W, C) for display
plt.show()

print(labels[0].item())


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\affel\\D7046E - Course\\ml-cloud-opt-thick-master\\Tjena\\data\\skogsstyrelsen\\skgs174cf5d2-46c7-ed11-9174-005056a6f472.nc'

In [80]:
import netCDF4
from netCDF4 import Dataset
for nc in 
    rootgrp = Dataset("test.nc", "w", format="NETCDF4")
    print(rootgrp.data_model)
    rootgrp.close()

['../data/skogsstyrelsen/2A-netcdfs-cropped-from-nuria/skgs_db7d323a-3fc7-ed11-9174-005056a6f472.nc'
 '../data/skogsstyrelsen/2A-netcdfs-cropped-from-nuria/skgs_c4ade691-44c7-ed11-9174-005056a6f472.nc'
 '../data/skogsstyrelsen/2A-netcdfs-cropped-from-nuria/skgs_4265b290-3ac7-ed11-9174-005056a6f472.nc'
 '../data/skogsstyrelsen/2A-netcdfs-cropped-from-nuria/skgs_dbd95f27-3ac7-ed11-9174-005056a6f472.nc'
 '../data/skogsstyrelsen/2A-netcdfs-cropped-from-nuria/skgs_fcdb2f62-43c7-ed11-9174-005056a6f472.nc'
 '../data/skogsstyrelsen/2A-netcdfs-cropped-from-nuria/skgs_cde9b181-37c7-ed11-9174-005056a6f472.nc'
 '../data/skogsstyrelsen/2A-netcdfs-cropped-from-nuria/skgs_210bf8a8-45c7-ed11-9174-005056a6f472.nc'
 '../data/skogsstyrelsen/2A-netcdfs-cropped-from-nuria/skgs_93702317-47c7-ed11-9174-005056a6f472.nc'
 '../data/skogsstyrelsen/2A-netcdfs-cropped-from-nuria/skgs_7055b1e2-39c7-ed11-9174-005056a6f472.nc'
 '../data/skogsstyrelsen/2A-netcdfs-cropped-from-nuria/skgs_f61efa16-38c7-ed11-9174-005056a

In [78]:
import netCDF4 as nc
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

# Define a custom dataset class
class CloudDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        
        # Read the NetCDF file and extract image data
        with nc.Dataset(file_path, 'r') as nc_file:
            # Assuming your image data is stored in a variable named 'image_data'
            image_data = nc_file.variables['image_data'][:]
        
        if self.transform:
            # Perform any necessary transformations
            image_data = self.transform(image_data)
            
        return image_data, label

# Define hyperparameters
num_classes = 2
batch_size = 32
learning_rate = 0.001
num_epochs = 10

# Define the neural network architecture
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create datasets and dataloaders
train_dataset = CloudDataset(train_file_paths, train_labels, transform=transforms.ToTensor())
val_dataset = CloudDataset(val_file_paths, val_labels, transform=transforms.ToTensor())
test_dataset = CloudDataset(test_file_paths, test_labels, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model, loss function, and optimizer
model = CNN(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Evaluate the model on the validation set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Validation Accuracy: {correct / total}")

# Evaluate the model on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Test Accuracy: {correct / total}")


NameError: name 'train_file_paths' is not defined

** DATASET AND SHAPE

In [60]:
# Load the datasets
train_gts = np.load('skogs_gts_train.npy')
val_gts = np.load('skogs_gts_val.npy')
test_gts = np.load('skogs_gts_test.npy')

train_json = np.load('skogs_json_train.npy', allow_pickle=True)
val_json = np.load('skogs_json_val.npy', allow_pickle=True)
test_json = np.load('skogs_json_test.npy', allow_pickle=True)

train_names = np.load('skogs_names_train.npy')
val_names = np.load('skogs_names_val.npy')
test_names = np.load('skogs_names_test.npy')



# GTS ÄR BINÄR DATA - TENSOR

skogs_gts_test_loader = torch.utils.data.DataLoader(train_gts)
skogs_gts_train_loader = torch.utils.data.DataLoader(test_gts)
skogs_gts_val_loader = torch.utils.data.DataLoader(val_gts)

# JSON ÄR METADATA - DOC {'Bilddatum': ['2019-09-19T00:00:00'], 'MedianvardeB2': tensor([0.0947]

skogs_json_test_loader =torch.utils.data.DataLoader(test_json)
skogs_json_train_loader =torch.utils.data.DataLoader(train_json)
skogs_json_val_loader = torch.utils.data.DataLoader(val_json)

# NAMES ÄR .NC FILER - LIST
skogs_names_test_loader =torch.utils.data.DataLoader(train_names)
skogs_names_train_loader = torch.utils.data.DataLoader(test_names)
skogs_names_val_loader =torch.utils.data.DataLoader(val_names)



# Convert to Torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
inputs_train = torch.Tensor(train_names).to(device)
inputs_val = torch.Tensor(val_names).to(device)
gts_train = torch.Tensor(train_gts).to(device)
gts_val = torch.Tensor(val_gts).to(device)

TypeError: can't convert np.ndarray of type numpy.str_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [97]:
# Första utkasst för att printa och fixa en matris, skriv gärna om
# Example: Skriv om så den hämtar alla labeles
y_true = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # Actual labels (1 for cloudy, 0 for not cloudy)
y_pred = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  # Predicted labels

# Create confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Calculate additional metrics
acc = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Print additional metrics
print(f"Accuracy (ACC): {acc:.4f}")
print(f"Precision (P): {precision:.4f}")
print(f"Sensitivity (Sn): {recall:.4f}")
specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
print(f"Specificity (Sp): {specificity:.4f}")
print(f"F-score: {f1:.4f}")


# Display confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Plot confusion matrix using seabornS



Accuracy (ACC): 0.5000
Precision (P): 0.5000
Sensitivity (Sn): 1.0000
Specificity (Sp): 0.0000
F-score: 0.6667
Confusion Matrix:
[[0 5]
 [0 5]]


# DATASET (i believe)

In [73]:
import netCDF4 as nc
import numpy as np
import os

# Assuming a base directory for the npy files, update this path as necessary
BASE_PATH_NPY = '/path/to/your/npy/files'

# Function to safely print the contents of a numpy array
def print_array_contents(data, limit=5):
    if data.size > limit:
        print(data[:limit])
        print(f"... (and {data.size - limit} more items)")
    else:
        print(data)

# Function to safely print the contents of a numpy array
def print_array_contents(data, limit=5):
    # If the data is too large, we only print a part of it
    if data.size > limit:
        print(data[:limit])
        print(f"... (and {data.size - limit} more items)")
    else:
        print(data)

# Function to print the contents of a netCDF file
def print_netcdf_contents(file_path):
    full_path = os.path.abspath(file_path)  # Get the absolute path of the file
    try:
        dataset = nc.Dataset(full_path, 'r')
        print(f"File: {full_path}")
        print(f"Dimensions: {list(dataset.dimensions.keys())}")
        print("Variables:")
        for var in dataset.variables.keys():
            print(f"  {var}:")
            print(f"    {dataset.variables[var]}")
        dataset.close()
    except Exception as e:
        print(f"Could not read file {full_path}: {e}")

# For npy files, assuming they're not netCDF files but you want to print their paths
npy_file_names = [
    'skogs_gts_test.npy',
    'skogs_gts_train.npy',
    'skogs_gts_val.npy',
    'skogs_json_test.npy',
    'skogs_json_train.npy',
    'skogs_json_val.npy',
    'skogs_names_test.npy',
    'skogs_names_train.npy',
    'skogs_names_val.npy'
]

# Print paths for npy files (assuming they're not meant to be read as netCDF files)
for file_name in npy_file_names:
    full_path = os.path.join(BASE_PATH_NPY, file_name)
    print(f"Full path for {file_name}: {full_path}")

print("\n" + "-"*50 + "\n")


# List of relative or absolute netCDF file paths
nc_file_paths = [
    '2A-netcdfs-cropped-from-nuria/skgs_0b5101fb-44c7-ed11-9174-005056a6f472.nc',
    '2A-netcdfs-cropped-from-nuria/skgs_0ba812a4-39c7-ed11-9174-005056a6f472.nc',
    '2A-netcdfs-cropped-from-nuria/skgs_0d455c60-44c7-ed11-9174-005056a6f472.nc',
    '2A-netcdfs-cropped-from-nuria/skgs_0d067942-3cc7-ed11-9174-005056a6f472.nc'
]

# Apply the function to each netCDF file
for nc_path in nc_file_paths:
    print_netcdf_contents(nc_path)
    print("\n" + "-"*50 + "\n")  # Separator for readability


Full path for skogs_gts_test.npy: /path/to/your/npy/files/skogs_gts_test.npy
Full path for skogs_gts_train.npy: /path/to/your/npy/files/skogs_gts_train.npy
Full path for skogs_gts_val.npy: /path/to/your/npy/files/skogs_gts_val.npy
Full path for skogs_json_test.npy: /path/to/your/npy/files/skogs_json_test.npy
Full path for skogs_json_train.npy: /path/to/your/npy/files/skogs_json_train.npy
Full path for skogs_json_val.npy: /path/to/your/npy/files/skogs_json_val.npy
Full path for skogs_names_test.npy: /path/to/your/npy/files/skogs_names_test.npy
Full path for skogs_names_train.npy: /path/to/your/npy/files/skogs_names_train.npy
Full path for skogs_names_val.npy: /path/to/your/npy/files/skogs_names_val.npy

--------------------------------------------------

File: /Users/ludvigflodin/Code/ml-cloud-opt-thick-master/Tjena/2A-netcdfs-cropped-from-nuria/skgs_0b5101fb-44c7-ed11-9174-005056a6f472.nc
Dimensions: ['time', 'y', 'x']
Variables:
  time:
    <class 'netCDF4._netCDF4.Variable'>
float64 

# MODEL?

In [47]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os

# Adjust the base path to your dataset's location
BASE_PATH_DATA = r'../data/'


# Function to load image paths from a .npy file
def load_image_paths(file_name):
    return np.load(os.path.join(BASE_PATH_DATA, file_name), allow_pickle=True)

# Function to visualize images
def visualize_images(image_paths, title='Images', num_images=5):
    plt.figure(figsize=(15, 3))
    plt.suptitle(title)
    for i in range(min(num_images, len(image_paths))):
        img_path = os.path.join(BASE_PATH_DATA, image_paths[i])
        img = xr.open_dataset(img_path).to_array().mean(dim=0)  # Assuming multi-band images; using the mean for visualization
        plt.subplot(1, num_images, i + 1)
        plt.imshow(img, cmap='gray')
        plt.axis('off')
    plt.show()

# Assuming 'skogs_names_test.npy' contains relative paths to the images
image_paths_test = load_image_paths('skogs_names_test.npy')

# Visualize 5 images from the test dataset
visualize_images(image_paths_test, title='Test Images')


NotADirectoryError: [Errno 20] Not a directory: 'skogs_names_test.npy/skogs_names_test.npy'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchvision.models as models

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 13 * 13, 120)
        self.fc2 = nn.Linear(120, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
# Implement a train model function so you can re_use it in task 3 and 4. 
# Should return the best performing model after training
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    best_val_loss = float('inf')
    best_model = None
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.cuda(), labels.cuda()
                outputs = model(inputs)
            
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
        
        val_loss /= len(val_loader.dataset)
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict()
    
    # Load the best model state
    model.load_state_dict(best_model)
    return model


# Hyperparams. Set these to reasonable values
BATCH_SIZE = ...
SHUFFLE = ...
LEARNING_RATE = ...



# Move data and model to GPU (CUDA)

In [ ]:
BATCH_SIZE = ...
SHUFFLE = ...
LEARNING_RATE = ...


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)

for inputs, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    ...

# Training loop

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0